In [1]:
import holmes.experiment as Ex
import tempfile
import pprint
import json
import os
from terrarium.flatten import flatten_dict, expand_dict
import tempfile

In [ ]:
temp_dir_obj = tempfile.TemporaryDirectory()
temp_dir = temp_dir_obj.name
package_location = Ex.staging.prep_holmes_package(
    staging_path=temp_dir, 
    commit_hash='ca6ccd63f270d603ffa9bdf5a7ced184d6339796'
)

detects_dict={}
for video_data in Ex.val_data:
    detects_dict[video_data['video_id']] = video_data['video_path'].replace('processed','detections')

def_conf_path = Ex.staging.extract_default_config(package_location, temp_dir)

with open(def_conf_path, 'r') as fp:
    def_conf_dict = json.load(fp)

new_def_conf_dicts={}
list_dicts=[]
for video_data in Ex.val_data:
    new_def_conf_dict = def_conf_dict.copy()
    detects_path = video_data['video_path'].replace('processed','detections').replace('.mp4', '.json')
    new_def_conf_dict['detector_config'] = {"source_path": detects_path,"_class_name": "PassThroughDetectorConfig","batch_size": 1}
    new_def_conf_dict['video_config'] = video_data
    new_def_conf_dict['video_config']['_class_name'] = 'RawVideoConfig'
    list_dicts.append(new_def_conf_dict)

In [10]:
measurement_ids = ['M01-PS0304-001', 'M02-PP0214-000', 'M00-ChvUtd-000', 'M03-ER0429-001']
hungarian_threshold = [10,28,46,64,82,100]

In [ ]:
list_configs=[]
for ht in hungarian_thresholds:
    for config in list_dicts:
        current_config_dict = config.copy()
        parameter_map = {"tracker_config.associator_config._class_name": 'SpatialAssociatorConfig', "tracker_config.associator_config.threshold": ht}
        flat_dict = flatten_dict(current_config_dict)
        flat_dict.update(parameter_map)
        updated_config_dict = expand_dict(flat_dict)

        file_name = os.path.join(temp_dir, "M{:02d}-{}-{:02d}.json".format(len(list_configs), 
                                            current_config_dict['video_config']['video_id'], ht))
        list_configs.append(file_name)
        with open(file_name, 'w') as f:
            json.dump(updated_config_dict, f, indent=2)
print(len(list_configs), list_configs[0])

In [ ]:
experiment = Ex.AWSExperiment(
    experiment_id="PTA-95-HT-1-1", job_queue=Ex.AWSComputeQueue.cpu_queue
)

for i, config_path in enumerate(list_configs):
    meas_id = os.path.basename(config_path).replace(".json", "")
    experiment.add_measurement(
        Ex.AWSMeasurement(
            experiment_id = "PTA-95-HT-1-1",
            measurement_id = meas_id,
            job_queue = Ex.AWSComputeQueue.cpu_queue,
            package_path = package_location,
            config_path = config_path
        )
    )

In [13]:
import boto3
experiment._batch_client = boto3.client('batch', region_name='us-east-1')

In [ ]:
experiment.stage_experiment()

In [3]:
experiment.launch_experiment()

In [1]:
import pandas as pd

In [11]:
data = pd.read_csv('/Users/jason.katz/Downloads/hung3.csv')

In [12]:
from bokeh.charts import Scatter
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row

In [13]:
output_notebook()

Loading BokehJS ...

In [ ]:
data['ht'] = 1
for index, row in data.iterrows():
    data['ht'][index] = row['job_id'][-3:]
data['ht'][data['ht']<0]=data['ht'][data['ht']<0]*-1

In [ ]:
data['game'] = 'id'
for index, row in data.iterrows():
    data['game'][index] = row['job_id'][20:26]

In [16]:
scat1 = Scatter(data, x='ht', y='mota', color='game')
show(scat1)

In [17]:
data_ht = data.copy()
data_ht['hungarian_threshold'] = data_ht['ht']
data_ht = data_ht.groupby(['ht']).mean()
data_ht

,mota,motp,motr,mostly_tracked,mt_auc,hungarian_threshold
ht,,,,,,
10,0.672471,0.649297,0.733826,0.432157,0.727345,10
28,0.710704,0.642245,0.782900,0.550912,0.776123,28
46,0.714994,0.641175,0.789975,0.565035,0.782787,46
64,0.718542,0.640535,0.795641,0.585925,0.788023,64
82,0.720653,0.640163,0.799665,0.597462,0.791817,82
100,0.722046,0.639926,0.802513,0.607699,0.794376,100


In [18]:
scat2 = Scatter(data_ht, x = 'hungarian_threshold', y='mota')
scat3 = Scatter(data_ht, x = 'hungarian_threshold', y='motp')
scat4 = Scatter(data_ht, x = 'hungarian_threshold', y='mostly_tracked')
scat5 = Scatter(data_ht, x = 'hungarian_threshold', y='mt_auc')
from bokeh.layouts import column, row
show(column(row(scat2, scat3), row(scat4, scat5)))